In [3]:
#Requierd Libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

PermissionError: [Errno 1] Operation not permitted

# Stroke Predciton Data Preprocessing

In [4]:
#Function used to load csv data given a path 
def load_data(data_path):
    return pd.read_csv(data_path)

In [ ]:
#Loading data and generating a data frame
data_path = "../data/healthcare-dataset-stroke-data.csv"
stroke_data = load_data(data_path)
stroke_df = pd.DataFrame(stroke_data)

In [ ]:
stroke_df

In [ ]:
#Generating the Descriptive Statistic
stroke_df.describe()

 <font size = 3> 
Observations: <br>
  - The average age is 43 years<br>
  - The average bmi is close to 29, which is considered unhealthy<br>
  - The average glucose level is 106, which is slightly unhealthy <br> 
  - The average in both hypertension and heart disease is low, meaning the population sample have a healthy hearth<br>
  - The 25%, 50%, 75% for hypertension, heart_disease, and stroke is 0. Meaning a most of the <br>
    patients in the Data set did not have a heart disease, hypertension or a stroke

In [ ]:
#Ploting stroke frequency for diffrent classes
stroke_class_list = ['gender','hypertension', 'heart_disease', 
                     'ever_married','work_type', 
                     'Residence_type','smoking_status'] 

plt.figure(figsize = (20, 15))
sns.set_theme(style="whitegrid", palette="muted")

for i in enumerate(stroke_class_list):
        plt.subplot(3, 3,i[0]+1)
        sns.countplot(data =stroke_df, x = i[1], hue = "stroke")
        plt.xticks(rotation = 10)

In [ ]:
#Distribution plot age vs stroke
sns.set_theme(style="whitegrid", palette="bright")
plt.figure(figsize = (20, 15))
dis_plt = sns.displot(data = stroke_df, x = "age", hue = "stroke" )

<font size = 4>
Obesravtions:<br> 
 - Most patience did nor have a stroke<br>
 - Patience over the age of 40 are more porne to having a stroke<br>
 - Patience aroung the age of 80 have had the most amount of strokes

In [ ]:
# Violin plot to understand the patients likelihood of getting a stroke
stroke_violin = sns.violinplot(data = stroke_df, x = "stroke", y = "age")
stroke_violin.set_title("Stroke vs Age, Violin Plot")

<font size = 4>
Obesravtions:<br> 
 - The highest number of strokes is for patience close to the age of 80 years 

In [ ]:
#count plot to check if dataset is balanced
countplot = sns.countplot(data = stroke_df, x = "stroke") 
countplot.set_title("Count Stroke vs No Sroke")

- <font size = 4>  As we can see from the plot, there is a significant number of target values, of people that did not have a stroke compared to the target values of people with a stroke, this tells us that the dataset is imbalanced.

In [ ]:
#Generating a heat map to uderstand the correlation among variables
labels = ["gender","age", "hypertension" , "heart_disease", "ever_married" , "work_type", "Residence_type", "avg_glucose_level", "bmi", "smoking_status","stroke"]
sns.heatmap(data = stroke_df[labels].corr(), annot=True,fmt=".1f", vmax = 1, cmap="YlGnBu" ).set_title("Stroke Data Correlation")

<font size = 4>
Obesravtions:<br> 
 - Hypertension, heart disease, bmi, age, average glucose levels have are strongly coreleated.

### Removing Outliers

In [ ]:
#Function for ploting a box plot given a DF and features
def plot_boxplot(df, ft):
    df.boxplot(column = ft)
    plt.grid(False)
    plt.show()

In [ ]:
#Ploting box plot of bmi and avg_glucose_level to check for outliers
features = ["bmi","avg_glucose_level"]
plot_boxplot(stroke_df, features)

In [ ]:
#Function to check for outliers, and return a list of idecies
def outliers(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3-Q1
    lowerBound = Q1 -1.5*IQR
    upperBound = Q3 +1.5*IQR
    index_list = df.index[(df[ft] < lowerBound) | (df[ft] > upperBound)]
    return index_list

In [ ]:
#Storing indecies of outliers
index_list = []
for ft in ["bmi","avg_glucose_level"]:
    index_list.extend(outliers(stroke_df, ft))

In [ ]:
#Function to remove outliers given index of outlier
def removeOutliers(df, index_list): 
    #Sorting indeces and making them unique
    index_list = sorted(set(index_list))
    df = df.drop(index_list)
    return df

In [ ]:
#Removing outliers for bmi and avg_glucose_level, and creating a new df
stroke_df_cleaned = removeOutliers(stroke_df, index_list)
stroke_df_cleaned.shape

### Handelling Null Values

In [ ]:
#Checking for null values in the Data set
stroke_df_cleaned.isnull().sum()

In [ ]:
#Handeling null values for BMI feature, Using Mean imputation
bmi_mean = stroke_df_cleaned['bmi'].mean()
stroke_df_cleaned['bmi'] = stroke_df_cleaned['bmi'].fillna(bmi_mean)

In [ ]:
#Checking for null values in the Data set
stroke_df_cleaned.isnull().sum()

### Encoding Objects 

In [ ]:
#Checking Data Types before encoding
stroke_df_cleaned.dtypes

In [ ]:
#Encoding all object data, in the data frame
obj_columns = ["gender ","ever_married ","work_type","Residence_type ","smoking_status"]
encoded_stroke_df = pd.get_dummies(stroke_df_cleaned,obj_columns,drop_first=True)
encoded_stroke_df.head()

In [ ]:
#Checking Data Types after encoding
encoded_stroke_df.dtypes

In [ ]:
#Saving new datset
directory_path = "../data/"
encoded_stroke_df.to_csv(directory_path + "preprocessed-healthcare-dataset-stroke-data.csv", index = "False")